In [1]:
from transformers import AutoModel, AutoTokenizer, EncoderDecoderModel, BertTokenizer, AdamW, BertModel
from datasets import load_dataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import math

In [2]:
raw = load_dataset('paws', 'labeled_final')

Reusing dataset paws (C:\Users\Pranav\.cache\huggingface\datasets\paws\labeled_final\1.1.0\09d8fae989bb569009a8f5b879ccf2924d3e5cd55bfe2e89e6dab1c0b50ecd34)


In [3]:
raw['train']

Dataset({
    features: ['id', 'sentence1', 'sentence2', 'label'],
    num_rows: 49401
})

In [3]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-cased', 'bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationshi

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [5]:
def remove_non_paraphrases(example):
    if example['label'] == 1:
        return example
    return None

In [6]:
paraphrases = raw.map(remove_non_paraphrases, batched=False)

In [7]:
paraphrases = raw.filter(lambda example: example['label'] == 1)

Loading cached processed dataset at C:\Users\Pranav\.cache\huggingface\datasets\paws\labeled_final\1.1.0\09d8fae989bb569009a8f5b879ccf2924d3e5cd55bfe2e89e6dab1c0b50ecd34\cache-f8f4b05c63816056.arrow
Loading cached processed dataset at C:\Users\Pranav\.cache\huggingface\datasets\paws\labeled_final\1.1.0\09d8fae989bb569009a8f5b879ccf2924d3e5cd55bfe2e89e6dab1c0b50ecd34\cache-9bfc9aaee2aa1e33.arrow
Loading cached processed dataset at C:\Users\Pranav\.cache\huggingface\datasets\paws\labeled_final\1.1.0\09d8fae989bb569009a8f5b879ccf2924d3e5cd55bfe2e89e6dab1c0b50ecd34\cache-274702c64a43f78c.arrow


In [8]:
paraphrases

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 21829
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 3536
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 3539
    })
})

In [9]:
def tokenize(examples):
    return tokenizer(examples['sentence1'], padding='max_length', truncation=True)

In [10]:
tokenized_data = paraphrases.map(tokenize, batched=True)

In [11]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

In [12]:
tokenized_data['train']

Dataset({
    features: ['attention_mask', 'id', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
    num_rows: 21829
})

In [13]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

device(type='cuda')

In [14]:
model.to(device)

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [15]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [16]:
len(paraphrases['train'])

21829

In [17]:
train_loader = DataLoader(paraphrases['train'], shuffle=True, batch_size=8)
eval_loader = DataLoader(paraphrases['test'], shuffle=True, batch_size=3)

In [18]:
for batch in train_loader:
    print(batch)
    break

{'id': tensor([21557,  1147, 45465, 45116,  1312, 48417, 12813, 21526]), 'sentence1': ["Femi 's musical career began when he started playing in the band of his father , Egypt 80 .", "Early charges used the term `` obscenity '' as well as after `` Miller v. California '' , though the term `` pornography '' remained as a reference entry :", 'Both traced the continuing influence on cultural studies of the kinds of literary and cultural materialism developed by Williams and his successors .', 'Atenaide ( RV 702 ) is a 1728 opera by Vivaldi for a revised edition of a libretto from 1709 by Caldara for Apostolo Zeno .', 'It is equivalent to the rank of rear admiral in the United States Navy and the rank of a rear admiral ( upper half ) in the Royal Navy .', 'Garentiyadangi is a village in the Bhopal district of Madhya Pradesh , India . It is located in the Berasia tehsil .', 'Bridie Morton married Timothy Torlot in 1986 .', 'The association of the stylised eye with mirrors was so strong that 

In [19]:
%load_ext line_profiler
epoch_bar = tqdm(range(10))
num_batches = math.ceil(len(paraphrases['train'])/8)

def train():
    for epoch in range(10):
        progress_bar = tqdm(range(num_batches))
        for batch in train_loader:
            input_ids = tokenizer(batch['sentence1'], return_tensors="pt", padding=True).input_ids.to(device)
            labels = tokenizer(batch['sentence2'], return_tensors="pt", padding=True).input_ids.to(device)
            outputs = model(input_ids=input_ids, decoder_input_ids=labels, labels=labels)
            optimizer.zero_grad()
            outputs.loss.backward()
            optimizer.step()
            progress_bar.update(1)

        model.eval()
        for batch in eval_loader:
            input_ids = tokenizer(batch['sentence1'], return_tensors="pt", padding=True).input_ids.to(device)
            labels = tokenizer(batch['sentence2'], return_tensors="pt", padding=True).input_ids.to(device)
            with torch.no_grad():
                generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder.start_token_id)
                print(tokenizer.decode(input_ids[0]))
                print(tokenizer.decode(generated[0]))
            break
        epoch_bar.update(1)

In [20]:
%lprun -f train train()

*** KeyboardInterrupt exception caught in code being profiled.

In [148]:
model.eval()
for batch in eval_loader:
    input_ids = tokenizer(batch['sentence1'], return_tensors="pt", padding=True).input_ids.to(device)
    with torch.no_grad():
        generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder_start_token_id)
    print(tokenizer.decode(input_ids[0]))
    print(tokenizer.decode(generated[0]))
    break

[CLS] Companies based in the larger Rivium include Rockwell Automation, Pfizer Nederland, Sodexho Nederland, Royal Dutch Shell, Van Oord, and many more. [SEP]
[CLS] Company companies headquartered in the larger Rivium Automation, Pfizer Nedland, So


In [151]:
tok = BertTokenizer.from_pretrained('bert-base-uncased')

In [164]:
enc = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [165]:
enc_in = tokenizer("What?!", return_tensors='pt')

In [166]:
enc(**enc_in).last_hidden_state.shape

torch.Size([1, 5, 768])